In [1]:
import ee
import geemap

In [2]:
ee.Initialize()

In [3]:
from datetime import datetime

In [4]:
Map = geemap.Map()

In [5]:
def sen2_print(s2_collection):
  '''
  prints the important properites of sentienel 2 data collection
  '''
  print('CLOUDY_PIXEL_PERCENTAGE  :',s2_collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo())
  print('VEGETATION_PERCENTAGE    :',s2_collection.aggregate_array('VEGETATION_PERCENTAGE').getInfo())
  print('NOT_VEGETATED_PERCENTAGE :',s2_collection.aggregate_array('NOT_VEGETATED_PERCENTAGE').getInfo())

  # print('SENSOR_QUCLOUD_COVERAGE_ASSESSMENTALITY :',s2_collection.aggregate_array('CLOUD_COVERAGE_ASSESSMENT').getInfo())
  # print('GENERATION_TIME          :',s2_collection.aggregate_array('GENERATION_TIME').getInfo())
  # print('SENSING_ORBIT_NUMBER     :',s2_collection.aggregate_array('SENSING_ORBIT_NUMBER').getInfo())
  print('NODATA_PIXEL_PERCENTAGE  :',s2_collection.aggregate_array('NODATA_PIXEL_PERCENTAGE').getInfo())
  # print('DATATAKE_TYPE            :',s2_collection.aggregate_array('DATATAKE_TYPE').getInfo())
  # print('SENSING_ORBIT_NUMBER     :',s2_collection.aggregate_array('SENSING_ORBIT_NUMBER').getInfo())
  print('SNOW_ICE_PERCENTAGE      :',s2_collection.aggregate_array('SNOW_ICE_PERCENTAGE').getInfo())
  # print('THIN_CIRRUS_PERCENTAGE   :',s2_collection.aggregate_array('THIN_CIRRUS_PERCENTAGE').getInfo())
  print('WATER_PERCENTAGE         :',s2_collection.aggregate_array('WATER_PERCENTAGE').getInfo())
  print('Date                     :',milsec2date(s2_collection.aggregate_array('system:time_start').getInfo()))
  # print('system:band_names        :',s2_collection.aggregate_array('system:band_names').getInfo())

In [6]:
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

In [7]:
import rois

In [8]:
from rois import ROIS
rois = ROIS()
date_range = ('2017-05-20','2017-08-10')
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(date_range[0],date_range[1]) \
    .filterBounds(rois.abudhabi)
print(s2.size().getInfo())

48


In [9]:
date_range = ('2017-05-20','2017-08-10')
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                      .filterDate(date_range[0], date_range[1]) \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                      .filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE',10)) \
                      .filterBounds(rois.rio)
print(s2.size().getInfo())

0


In [17]:
import rois
rois = rois.ROIS()

for name, roi in rois:
  print('ROI name:',name)
  date_range = ('2017-01-01','2017-12-31')
  s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
                        .filterDate(date_range[0], date_range[1]) \
                        .filterBounds(roi)#.filter(ee.Filter.contains('.geo', roi)) 
  # s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
  #                       .filterDate(date_range[0], date_range[1]) \
  #                       .filterBounds(roi)
  print(s2.size().getInfo())


ROI name: abudhabi
248
ROI name: abudhabi_buffed
248
ROI name: aguasclaras
80
ROI name: aguasclaras_buffed
162
ROI name: aguasclaras_adj_goiania
31
ROI name: beihai
64
ROI name: beihai_buffed
64
ROI name: beihai_adj_gaoqiazhen
64
ROI name: beirut
54
ROI name: beirut_adj_tripoli
107
ROI name: bercy
106
ROI name: bercy_buffed_paris
106
ROI name: bordeaux
53
ROI name: bordeaux_buffed
100
ROI name: brasilia
108
ROI name: brasilia_buffed
216
ROI name: chongqing
60
ROI name: chongqing_adj
61
ROI name: cupertino
52
ROI name: cupertino_buffed
104
ROI name: cupertino_adj_losangles
108
ROI name: dubai
54
ROI name: dubai_buffed
162
ROI name: dubai_adj_sharje
107
ROI name: hongkong
202
ROI name: hongkong_adj
202
ROI name: lasvegas
101
ROI name: lasvegas_buffed
101
ROI name: milano
53
ROI name: milano_buffed
106
ROI name: montpellier
53
ROI name: montpellier_buffed
53
ROI name: mumbai
29
ROI name: mumbai_buffed
58
ROI name: nantes
208
ROI name: nantes_buffed
210
ROI name: norcia
52
ROI name: paris


In [31]:
# Function to get the Ratio of ones to total pixels
def get_mask_ones_ratio(mask:ee.Image, scale = 100, in_percentage = True, rescale_attempts = 5):
    """
    Function to get the percentage or the ratio of ones to total pixels in an Earth Engine image mask.

    Args:
    -----
        `mask` (ee.Image): An Earth Engine image mask.
        `scale` (int, optional): The scale to use for reducing the image. Defaults to 30.
        `in_percentage` (bool, optional): Whether to return the ratio or the percentage. Defaults to True.
        `rescale_attempts` (int, optional): Number of times to rescale the mask if reducer raised a too many pixel error.

    Returns:
    --------
        float: The ratio of ones to total pixels in the mask.
    """
    # Compute the number of ones and total number of pixels in the mask
    #band_name = mask.bandNames().getInfo()[0]
    
    for i in range(rescale_attempts):
        try:
            print('trying..')
            stats = mask.reduceRegion(
                reducer=ee.Reducer.sum().combine(
                    reducer2=ee.Reducer.count(),
                    sharedInputs=True
                ),
                geometry=mask.geometry(),
                scale=scale,
                maxPixels=1e9)
            stats.getInfo()
            print('try ended')
            break
        except:
            scale = scale * 10
            print(f"Scale was too Small -> Rescaling mask to {scale}m ")
            
    # Extract the number of ones and total number of pixels from the result
    print('we here')
    ones = stats.get(stats.keys().get(1))
    total = stats.get(stats.keys().get(0))

    # Compute the ratio of ones to total pixels
    ratio = ee.Number(ones).divide(total)
    

    # Return the ratio
    return ratio.multiply(100) if in_percentage else ratio


# Function to get the Ratio of Nulls to total pixels that an roi could have
def get_not_nulls_ratio(image:ee.Image, roi:ee.Geometry ,scale = 100, in_percentage = True, rescale_attempts = 5) -> ee.Number:
    """
        Calculates the ratio of not null null values to total pixels that an ROI (Region of Interest) could have for a given image.
        
        Args:
        -----
        - image (ee.Image): The image for which the nulls ratio needs to be calculated.
        - roi (ee.Geometry): The region of interest for which the nulls ratio needs to be calculated.
        - scale (int, optional): The scale at which to perform the reduction. Defaults to 30.
        
        Returns:
        --------
        - ratio (ee.Number): The ratio of not null null values to total pixels for the given ROI and image.
    """

    # Creates a 1 & 0 mask of the image, 0 on null areas, and 1 for pixels with values
    # th clip is really important since, mask() method goes over boundries.
    mask = image.mask().select(0).clip(roi)
    # Return the ratio
    return get_mask_ones_ratio(mask, scale = scale, in_percentage = in_percentage, rescale_attempts = rescale_attempts)

In [32]:
img = s2.first()


scale = 10
for i in range(10):
    try:
        img_pix_dict = img.reduceRegion(reducer=ee.Reducer.count(),geometry= roi,scale=scale).getInfo()
        break
    except:
        print(f'Attempt {i} with Scale of {scale} failed. Trying again with scale of {scale*10} ')
        scale = scale*10
        

trying..
try ended
we here


EEException: Image.reduceRegion: Too many pixels in the region. Found 35935789891, but maxPixels allows only 1000000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.

In [11]:
date_range1 = ('2017-05-20','2017-08-10')
date_range2 = ('2021-05-20','2021-08-10')
roi = rois.bercy
SR = False
if SR:
    print('Finding SR images')
    s2_1 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                        .filterDate(date_range1[0], date_range1[1]) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                        .filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE',10)) \
                        .filterBounds(roi).sort('CLOUDY_PIXEL_PERCENTAGE', True)
    print(s2_1.size().getInfo())

    s2_2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                        .filterDate(date_range2[0], date_range2[1]) \
                        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                        .filter(ee.Filter.lt('SNOW_ICE_PERCENTAGE',10)) \
                        .filterBounds(roi).sort('CLOUDY_PIXEL_PERCENTAGE', True)
    print(s2_2.size().getInfo())
else:
    print('Finding TOA images')
    s2_1 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(date_range1[0],date_range1[1]) \
        .filterBounds(roi).sort('CLOUDY_PIXEL_PERCENTAGE', True)
    print(s2_1.size().getInfo())
    
    s2_2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(date_range2[0],date_range2[1]) \
        .filterBounds(roi).sort('CLOUDY_PIXEL_PERCENTAGE', True)
    print(s2_2.size().getInfo())
        

Finding TOA images
23
33


In [12]:
vis_params_s2 = {
  'min': 0.04,
  'max': 0.4,
  'bands': ['B4', 'B3', 'B2'],
}

In [13]:
left_layer = geemap.ee_tile_layer(s2_1.first().clip(roi).multiply(0.0001), vis_params_s2, 'S2_1')
right_layer = geemap.ee_tile_layer(s2_2.first().clip(roi).multiply(0.0001), vis_params_s2, 'S2_2')

In [14]:
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [15]:
sen2_print(s2_2.sort('CLOUDY_PIXEL_PERCENTAGE', False))

CLOUDY_PIXEL_PERCENTAGE  : [100, 100, 99.9691, 99.8747, 99.6752, 99.2618, 98.5313, 95.1893, 92.2023, 90.6016, 90.0605, 87.9752, 87.8637, 86.353, 82.8295, 76.2768, 75.1655, 71.0818, 69.9768, 69.2295, 68.5597, 67.0051, 65.6219, 53.8, 53.2012, 45.4239, 41.6293, 38.1371, 10.0658, 4.2792, 1.6607, 0.0282, 0]
VEGETATION_PERCENTAGE    : []
NOT_VEGETATED_PERCENTAGE : []
NODATA_PIXEL_PERCENTAGE  : []
SNOW_ICE_PERCENTAGE      : []
WATER_PERCENTAGE         : []


NameError: name 'milsec2date' is not defined

In [ ]:

# Define a Sentinel-2 Image
image = ee.Image('COPERNICUS/S2_SR/20210101T001239_20210101T001237_T55HPJ')

# Define the Parameters for FLAASH Atmospheric Correction
atm_corr = ee.Image('users/samsammurphy/public/FLAASH_parameters')
h2o = atm_corr.select('h2o')
o3 = atm_corr.select('o3')
aot = atm_corr.select('aot')

# Apply FLAASH Atmospheric Correction
corrected = ee.Image(ee.Algorithms.Landsat.simpleCloudScore(image)).select(['B2', 'B3', 'B4', 'B8', 'QA60']) \
    .divide(10000) \
    .reproject('EPSG:4326', None, 10) \
    .unmask() \
    .multiply(10000) \
    .uint16() \
    .correctLandsat(atm_corr, 'TOA', 0.025)

# Display the Results
Map = geemap.Map()
Map.centerObject(image, 10)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Original Image')
Map.addLayer(corrected, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Corrected Image')
Map.addLayerControl()
Map


AttributeError: 'Image' object has no attribute 'correctLandsat'